In [1]:
from keras.layers import Bidirectional , GRU ,LSTM
import numpy as np
import pandas as pd


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

from keras.layers import Embedding

from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding , Convolution2D
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
from keras.models import Sequential

import numpy as np


Using Theano backend.
C:\Users\hp\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\hp\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import numpy as np
from gensim.models import KeyedVectors
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

import nltk
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from keras.layers import Embedding
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
import theano.ifelse
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from sklearn.model_selection import StratifiedKFold

## Loading data

In [3]:
train_data = pd.read_csv('C:/Users/hp/Desktop/IIITD/Medical Information Extraction/train_medical.csv')
test_data = pd.read_csv('C:/Users/hp/Desktop/IIITD/Medical Information Extraction/test_medical.csv')
validation_data = pd.read_csv('C:/Users/hp/Desktop/IIITD/Medical Information Extraction/validation_medical.csv')

In [4]:
#frames = [train_data , validation_data]
#train_data = pd.concat(frames)

In [5]:
#train_data.shape

In [6]:
columns_to_drop = ['_unit_id', '_created_at', '_canary', '_id', '_started_at', '_channel',
       '_trust', '_worker_id', '_country', '_region', '_city', '_ip',
       'direction', 'b1', 'b2', 'direction_gold', 'e1', 'e2',
       'relex_relcos', 'sent_id', 'twrex', 'term1' , 'term2']


train_data = train_data.drop( columns_to_drop , axis = 1)
test_data = test_data.drop(columns_to_drop , axis = 1)
validation_data = validation_data.drop(columns_to_drop , axis = 1)

train_data.loc[train_data.relation == 'is location of' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'diagnosed by' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'contraindicates' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'location' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'location of' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'diagnose_by_test_or_drug' , 'relation'] = 'Others'
train_data.loc[train_data.relation == 'is diagnosed by' , 'relation'] = 'Others'


test_data.loc[test_data.relation == 'is location of' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'diagnosed by' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'contraindicates' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'location' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'location of' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'diagnose_by_test_or_drug' , 'relation'] = 'Others'
test_data.loc[test_data.relation == 'is diagnosed by' , 'relation'] = 'Others'

validation_data.loc[validation_data.relation == 'is location of' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'diagnosed by' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'contraindicates' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'location' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'location of' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'diagnose_by_test_or_drug' , 'relation'] = 'Others'
validation_data.loc[validation_data.relation == 'is diagnosed by' , 'relation'] = 'Others'



In [7]:
print(train_data.shape , test_data.shape , validation_data.shape )

(13340, 2) (4566, 2) (4270, 2)


## Preprocessing with keras

In [13]:
train_data_list = [ x for x in train_data['sentence']]
test_data_list = [ x for x in test_data['sentence']]
validation_data_list = [ x for x in validation_data['sentence']]

In [14]:
## encoding the target variable
encoder = LabelEncoder()
encoder.fit(train_data['relation'])
y_train = encoder.transform(train_data['relation'])
y_test = encoder.transform(test_data['relation'])
y_val = encoder.transform(validation_data['relation'])

#y_train_cat = to_categorical(y_train)
#y_test_cat = to_categorical(y_test)

print(y_train.shape , y_test.shape , y_val.shape)

(13340,) (4566,) (4270,)


In [15]:
#tokenising
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data_list)

vocab_size = len(tokenizer.word_index) + 1

In [16]:
print(vocab_size)

7982


In [17]:
encoded_docs_train = tokenizer.texts_to_sequences(train_data_list)
encoded_docs_validation = tokenizer.texts_to_sequences(validation_data_list)
encoded_docs_test = tokenizer.texts_to_sequences(test_data_list)


In [18]:
# calculate max document length
lengths = [len(s.split()) for s in train_data_list]
max_length = max(lengths)
print(max_length)

97


In [19]:
##########padding
max_length = 97
padded_docs_train = pad_sequences(encoded_docs_train, maxlen = max_length, padding='post')
padded_docs_validation = pad_sequences(encoded_docs_validation, maxlen = max_length, padding='post')
padded_docs_test = pad_sequences(encoded_docs_test, maxlen = max_length, padding='post')


In [21]:
X_train = padded_docs_train
X_test = padded_docs_test
X_val = padded_docs_validation

In [22]:
print( X_train.shape , X_test.shape , X_val.shape)

(13340, 97) (4566, 97) (4270, 97)


In [28]:
y_train_cat = to_categorical(y_train)
y_val_cat = to_categorical(y_val)
y_test_cat = to_categorical(y_test)

## Word embeddings

In [23]:
filename_pmc = 'C:/Users/hp/Word_embeddings/PubMed-and-PMC-w2v.bin'
model_pmc = gensim.models.KeyedVectors.load_word2vec_format( filename_pmc  , binary=True)

In [24]:
EMBEDDING_DIM_PMC = 200
vocabulary_size =  vocab_size

In [25]:
embedding_matrix_pmc = np.zeros(( vocabulary_size , EMBEDDING_DIM_PMC ))
for word , i in tokenizer.word_index.items():
    try:
        embedding_vector_pmc = model_pmc[word]
    except KeyError:
        embedding_vector_pmc = None
    if embedding_vector_pmc is not None:
        embedding_matrix_pmc[i] = embedding_vector_pmc
        

## Multichannel CNN 

In [26]:
from keras.layers import Dense, Merge

In [32]:
from keras import backend as K

def precision(y_true, y_pred):
    """Precision metric.
     Only computes a batch-wise average of precision.
     Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
    """Recall metric.
     Only computes a batch-wise average of recall.
     Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [33]:
# channel 1
inputs1 = Input(shape=(max_length,))
embedding1 = Embedding(vocab_size, 200 ,weights=[embedding_matrix_pmc],
                          input_length = 97,
                          trainable = False )(inputs1)

conv1 = Conv1D(filters = 64, kernel_size = 4, activation='relu')(embedding1)
drop1 = Dropout(0.7)(conv1)
pool1 = MaxPooling1D(pool_size=2)(drop1)
flat1 = Flatten()(pool1)

# channel 2
inputs2 = Input(shape=(max_length,))
embedding2 = Embedding(vocab_size, 200 ,weights=[embedding_matrix_pmc],
                          input_length = 97,
                          trainable = False)(inputs2)

conv2 = Conv1D(filters=64, kernel_size=6, activation='relu')(embedding2)
drop2 = Dropout(0.7)(conv2)
pool2 = MaxPooling1D(pool_size=2)(drop2)
flat2 = Flatten()(pool2)
# channel 3
inputs3 = Input(shape=(max_length,))
embedding3 = Embedding(vocab_size,200 ,weights=[embedding_matrix_pmc],
                          input_length = 97,
                          trainable = False)(inputs3)
#x3 = Bidirectional(LSTM(128, return_sequences=True, dropout=0.10, recurrent_dropout=0.10))()
conv3 = Conv1D(filters=64, kernel_size=8, activation='relu')(embedding3)
drop3 = Dropout(0.7)(conv3)
pool3 = MaxPooling1D(pool_size=2)(drop3)
flat3 = Flatten()(pool3)

# merge
merged = concatenate([flat1, flat2, flat3])
# reshaping
reshape = Reshape(( 276 , 32 ))(merged)
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.10, recurrent_dropout=0.10))(reshape)
max_pool = GlobalMaxPooling1D()(x)

outputs = Dense(3, activation='softmax')(max_pool)
model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
# compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['acc' , precision , recall])
# summarize
print(model.summary())


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_4 (InputLayer)             (None, 97)            0                                            
____________________________________________________________________________________________________
input_5 (InputLayer)             (None, 97)            0                                            
____________________________________________________________________________________________________
input_6 (InputLayer)             (None, 97)            0                                            
____________________________________________________________________________________________________
embedding_6 (Embedding)          (None, 97, 200)       1596400                                      
___________________________________________________________________________________________

In [34]:
model.fit([X_train, X_train ,X_train], y_train_cat , epochs=10, batch_size=1000 ,  validation_data = ([X_val , X_val, X_val ], y_val_cat ))

Train on 13340 samples, validate on 4270 samples
Epoch 1/10
13340/13340 [==============================] - 787s - loss: 0.8644 - acc: 0.6019 - precision: 0.5280 - recall: 0.3996 - val_loss: 0.7326 - val_acc: 0.7173 - val_precision: 0.7666 - val_recall: 0.6307
Epoch 2/10
13340/13340 [==============================] - 5339s - loss: 0.5960 - acc: 0.7750 - precision: 0.7981 - recall: 0.7381 - val_loss: 0.6792 - val_acc: 0.7246 - val_precision: 0.7706 - val_recall: 0.6494
Epoch 3/10
13340/13340 [==============================] - 731s - loss: 0.5290 - acc: 0.7936 - precision: 0.8156 - recall: 0.7657 - val_loss: 0.6494 - val_acc: 0.7433 - val_precision: 0.7859 - val_recall: 0.6522
Epoch 4/10
13340/13340 [==============================] - 795s - loss: 0.4825 - acc: 0.8134 - precision: 0.8354 - recall: 0.7834 - val_loss: 0.6111 - val_acc: 0.7368 - val_precision: 0.7686 - val_recall: 0.6857
Epoch 5/10
13340/13340 [==============================] - 720s - loss: 0.4311 - acc: 0.8291 - precision: 0

In [35]:
scores = model.evaluate([X_test , X_test, X_test], y_test_cat)
print("%s : %.2f%%" % (model.metrics_names[1] , scores[1]*100))

4566/4566 [==============================] - 40s    
acc : 70.65%


In [36]:
print("%s : %.2f%%" % (model.metrics_names[2] , scores[2]*100))

precision : 71.63%


In [37]:
print("%s : %.2f%%" % (model.metrics_names[3] , scores[3]*100))

recall : 68.66%


In [38]:
precision = scores[2]*100
recall = scores[3]*100
## f1 score
f1_score = (2 * precision * recall )/( recall + precision )


In [39]:
f1_score

70.11577354271193